In [1]:
import pandas as pd
x_train = pd.read_csv('/kaggle/input/big-data-analytics-certification-kr-2022/train.csv')
x_test = pd.read_csv('/kaggle/input/big-data-analytics-certification-kr-2022/test.csv')
# print(x_train)

x_train.drop(columns = 'ID', inplace = True)
x_test_ID = x_test.pop('ID')

# print(x_train.columns)
# print(x_train.groupby(['Var_1', 'Segmentation'])['Segmentation'].count())

# 사용할 피처 : Ever_Married, Age, Graduated, Spending_Score, Family_Size
# 사용하지 않을 피처 : Gender, Profession, Work_Experience, Var_1
# 'Gender', 'Ever_Married', 'Age', 'Graduated', 'Profession', 
# 'Work_Experience', 'Spending_Score', 'Family_Size', 'Var_1'

y_train = x_train.pop('Segmentation')
# print(x_train, y_train)

x_train.drop(columns = ['Gender', 'Profession', 'Work_Experience', 'Var_1'], inplace = True)
x_test.drop(columns = ['Gender', 'Profession', 'Work_Experience', 'Var_1'], inplace = True)

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

x_train['Ever_Married'] = encoder.fit_transform(x_train['Ever_Married'])
x_train['Graduated'] = encoder.fit_transform(x_train['Graduated'])
x_train['Spending_Score'] = encoder.fit_transform(x_train['Spending_Score'])
x_test['Ever_Married'] = encoder.fit_transform(x_test['Ever_Married'])
x_test['Graduated'] = encoder.fit_transform(x_test['Graduated'])
x_test['Spending_Score'] = encoder.fit_transform(x_test['Spending_Score'])

from sklearn.preprocessing import RobustScaler, StandardScaler
scaler1 = RobustScaler()
scaler2 = StandardScaler()

x_train_object = x_train[['Ever_Married', 'Graduated', 'Spending_Score']]
x_train_int = x_train[['Age', 'Family_Size']]
x_test_object = x_test[['Ever_Married', 'Graduated', 'Spending_Score']]
x_test_int = x_test[['Age', 'Family_Size']]

x_train_object = pd.DataFrame(scaler1.fit_transform(x_train_object), columns = x_train_object.columns)
x_train_int = pd.DataFrame(scaler2.fit_transform(x_train_int), columns = x_train_int.columns)
x_test_object = pd.DataFrame(scaler1.fit_transform(x_test_object), columns = x_test_object.columns)
x_test_int = pd.DataFrame(scaler2.fit_transform(x_test_int), columns = x_test_int.columns)

x_train = pd.concat([x_train_object, x_train_int], axis = 1)
x_test = pd.concat([x_test_object, x_test_int], axis = 1)
# print(x_train)

from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2, random_state = 2022)

# # 1. 랜덤포레스트
# from sklearn.ensemble import RandomForestClassifier
# model1 = RandomForestClassifier(n_estimators = 200, max_depth = 5, random_state = 2022, criterion = 'gini')

# model1.fit(x_train, y_train)
# y_val_predict = pd.DataFrame(model1.predict(x_val))

# from sklearn.metrics import f1_score
# print(f1_score(y_val, y_val_predict,  average='macro'))

# n_estimators = 100, max_depth = 5, random_state = 2022, criterion = 'entropy' -> score : 0.4956802623069941
# n_estimators = 100, max_depth = 5, random_state = 2022, criterion = 'gini' -> score : 0.5078024027315914
# n_estimators = 200, max_depth = 5, random_state = 2022, criterion = 'gini' -> score : 0.5008460714188248

# # 2. xgboost
# from xgboost import XGBClassifier
# model2 = XGBClassifier(n_estimators = 8, max_depth = 4, random_state = 2022, criterion = 'entropy')

# model2.fit(x_train, y_train-1)
# y_val_predict = pd.DataFrame(model2.predict(x_val))

# from sklearn.metrics import f1_score
# print(f1_score(y_val, y_val_predict+1,  average='macro'))

# n_estimators = 200, max_depth = 5, random_state = 2022, criterion = 'entropy' -> score : 0.45329238005615924
# n_estimators = 200, max_depth = 5, random_state = 2022, criterion = 'gini' -> score : 0.45329238005615924
# n_estimators = 200, max_depth = 4, random_state = 2022, criterion = 'gini' -> score : 0.4638275202828763
# n_estimators = 100, max_depth = 4, random_state = 2022, criterion = 'entropy' -> score : 0.47087463454876255
# n_estimators = 8, max_depth = 4, random_state = 2022, criterion = 'entropy' -> score : 0.49357601645300975

# # 3. DecisionTree
# from sklearn.tree import DecisionTreeClassifier
# model3 = DecisionTreeClassifier(max_depth = 4, criterion = 'entropy', random_state = 2022)

# model3.fit(x_train, y_train)
# y_val_predict = pd.DataFrame(model3.predict(x_val))

# from sklearn.metrics import f1_score
# print(f1_score(y_val, y_val_predict,  average='macro'))

# max_depth = 5, criterion = 'gini', random_state = 2022 -> score : 0.4914928899481684
# max_depth = 4, criterion = 'gini', random_state = 2022 -> score : 0.49930817111423154
# max_depth = 4, criterion = 'entropy', random_state = 2022 -> score : 0.4997532690764852


# 검증 결과 랜포 n_estimators = 100, max_depth = 5, random_state = 2022, criterion = 'gini'
# 또는 의사결정나무 max_depth = 4, criterion = 'entropy', random_state = 2022
# 둘 중 하나로 결정


# 1. 랜포 n_estimators = 100, max_depth = 5, random_state = 2022, criterion = 'gini'
# from sklearn.ensemble import RandomForestClassifier
# model1 = RandomForestClassifier(n_estimators = 200, max_depth = 5, random_state = 2022, criterion = 'gini')

# model1.fit(x_train, y_train)
# y_test = pd.DataFrame(model1.predict(x_test))
# y_test.columns = ['Segmentation']

# result = pd.concat([x_test_ID, y_test], axis = 1)
# result.to_csv('Submission.csv', index = False)

# 2. 의사결정나무 max_depth = 4, criterion = 'entropy', random_state = 2022
from sklearn.tree import DecisionTreeClassifier
model3 = DecisionTreeClassifier(max_depth = 4, criterion = 'entropy', random_state = 2022)

model3.fit(x_train, y_train)
y_test = pd.DataFrame(model3.predict(x_test))
y_test.columns = ['Segmentation']

result = pd.concat([x_test_ID, y_test], axis = 1)
result.to_csv('Submission.csv', index = False)